# User Journey Cohort Analysis (Web Session Data)

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

The original raw dataset for this project actually comes from **Google Analytics (BigQuery)**. It tracks website session data across all user visits for an entire year. This dataset is massive, so only 7 columns were extracted from BigQuery to prevent performance issues. They will be imported into this notebook in the following cell.

In [ ]:
# Load data
google = pd.read_csv("google_merchandise_raw.csv")

In [ ]:
# Preview the dataset
print(google.head(10))

### Initial Data Checks

In [ ]:
# Number of rows and columns
print(google.shape)

In [ ]:
# Rename columns for clarity
google = google.rename(columns={
    'fullVisitorId': 'user_id',
    'visitId': 'session_id',
    'visitStartTime': 'session_start_time',
    'hitNumber': 'hit_number'
})

In [ ]:
# Check data types
print(google.dtypes)

In [ ]:
# Convert `session_start_time` from unix to datetime
google['session_start_time'] = pd.to_datetime(google['session_start_time'], unit='s')

In [ ]:
# Convert `hit_type` entries to lowercase
google['hit_type'] = google['hit_type'].str.lower()

In [ ]:
# Number of unique users
unique_users = google['user_id'].nunique()
print(f'Number of Unique Users: {unique_users}')

In [ ]:
# Number of unique sessions
unique_sessions = google['session_id'].nunique()
print(f'Number of Unique Sessions: {unique_sessions}')

In [ ]:
# Missing values by column
missing_values = google.isnull().sum()
print(f'Number of Missing Values (per column): \n{missing_values}')

In [ ]:
# Duplicate rows
duplicates = google[google.duplicated()]
print(f'Number of Duplicate Rows: {len(duplicates)}')

In [ ]:
# Date range
start = google['session_start_time'].min()
end = google['session_start_time'].max()
print(f'Date Range: {start} thru {end}')

### Initial cleaning done... ready for analysis

- **Rows:** 4,153,675
- **Columns:** 7
- **Unique Users:** 843,049
- **Unique Sessions:** 886,303
- **Missing Values:**
  - `revenue`: 4,141,602 🆗
- **Duplicate Rows:** 0 ✅
- **Date Range:** Aug 1, 2016 - Aug 2, 2017

The `revenue` column will **not** be used in this analysis, so these missing values are irrelevant.

# Analysis

### Purchase Rate by Path

In [ ]:
# Build path lists for every session
session_paths = (
    google.sort_values(['user_id', 'session_id', 'hit_number']).groupby(['user_id', 'session_id'])['page_path'].apply(list).reset_index(name='path')
)

In [ ]:
# Convert lists to strings
session_paths['path_strings'] = session_paths['path'].apply(lambda x: ' / '.join(x))

In [ ]:
# Number of unique paths
unique_path_count = session_paths['path_strings'].nunique()
print(f'Number of Unique Paths: {unique_path_count}')

In [ ]:
# Top 10 paths taken
top_paths = session_paths['path_strings'].value_counts().head(10)
print(top_paths)

In [ ]:
# Path frequency
path_frequency = session_paths['path_strings'].value_counts().reset_index()
path_frequency.columns = ['path', 'frequency']

In [ ]:
# Set a 'purchased' flag... either 'purchase' or 'no purchase'
purchased = google.groupby(['user_id', 'session_id'])['revenue'].max().notnull().astype(int).reset_index(name='purchased')

In [ ]:
# Merge with session_paths df
paths_with_purchase = session_paths.merge(purchased, on=['user_id', 'session_id'])

In [ ]:
# Find purchase rate for every path
path_stats = (
    paths_with_purchase.groupby('path_strings').agg(
        frequency=('path_strings', 'size'), purchases=('purchased', 'sum')).assign(
            purchase_rate=lambda df: df['purchases'] / df['frequency']
    ).reset_index()
)

In [ ]:
# Display top 10 paths according to purchase rate
print(path_stats.sort_values('purchase_rate', ascending=False).head(10))

**Note:** Overall, purchases are rare. The vast majority of paths are simply browsing and searching, etc. Therefore, only paths with **at least 1 purchase** will be explored next.

In [ ]:
# Filter for paths with at least 1 purchase
purchases_only = path_stats[path_stats['purchases'] >= 1].sort_values('purchases', ascending=False)

In [ ]:
# Preview paths with at least 1 purchase
print(purchases_only.head(10))

### Purchase Rate by Path Length

In [ ]:
# Calculate path length for every session
session_paths['path_length'] = session_paths['path'].apply(len)

In [ ]:
# Merge with 'purchased' flag
session_paths = session_paths.merge(purchased, on=['user_id', 'session_id'])

In [ ]:
# Path length value counts
print(session_paths['path_length'].value_counts().sort_index())

In [ ]:
# Histogram ^

plt.figure(figsize=(10,5))
plt.hist(session_paths['path_length'], bins=range(1, session_paths['path_length'].max()+2), edgecolor='cyan')
plt.title('Distribution of Path Lengths')
plt.xlabel('Path Length (Number of Steps)')
plt.ylabel('Number of Sessions')
plt.yscale('log')
plt.show()

**Note:** A **log scale** was applied to the y-axis here because as path length (number of steps) increases, the number of sessions has an extreme drop off. Most y-axis values are quite small, so the *very few* large y-axis values would dominate this histogram without the log application.

In [ ]:
# Put all path lengths into buckets
bins = [1, 2, 3, 4, 6, 11, 21, 51, session_paths['path_length'].max() + 1]
labels = ['1', '2', '3', '4-5', '6-10', '11-20', '21-50', '51+']

# New buckets column
session_paths['path_length_bucket'] = pd.cut(session_paths['path_length'], bins=bins, labels=labels, right=False)

In [ ]:
# Calculate purchase rate by bucket
purchase_by_bucket = (
    session_paths.groupby('path_length_bucket').agg(
        sessions=('path_length', 'count'), purchases=('purchased', 'sum')).assign(
        purchase_rate=lambda x: x['purchases'] / x['sessions']).reset_index()
)

In [ ]:
# Find % of total purchases
total_purchases = purchase_by_bucket['purchases'].sum()

purchase_by_bucket['pct_of_total_purchases'] = (purchase_by_bucket['purchases'] / total_purchases * 100).round(2)

In [ ]:
# Display purchase rate by bucket
print(purchase_by_bucket)

# Plot results
plt.figure(figsize=(8,5))
plt.bar(purchase_by_bucket['path_length_bucket'], purchase_by_bucket['purchase_rate'])
plt.title('Purchase Rate by Path Length Bucket')
plt.xlabel('Path Length Bucket')
plt.ylabel('Purchase Rate')
plt.ylim(0, purchase_by_bucket['purchase_rate'].max() * 1.1)  # adjusted scale
plt.grid(axis='y')
plt.show()

**Results:** Sessions with **more steps** (i.e., longer user journeys) are far more likely to **result in a purchase** than shorter sessions.

### Drop-Off % at Each Step

In [ ]:
# Calculate number of sessions that reach each step number (i.e., Step 1, Step 2, ...), generally speaking
session_lengths = session_paths['path_length']
max_length = session_lengths.max()

In [ ]:
# For each step n, count number of sessions that made it to step n or further
sessions_at_least_n = [(session_lengths >= n).sum() for n in range(1, max_length + 1)]

In [ ]:
# Calculate drop-off percentage between steps
dropoff_pct = [
    1 - (sessions_at_least_n[n] / sessions_at_least_n[n-1]) if n > 0 else 0 for n in range(len(sessions_at_least_n))
]

In [ ]:
# Create DataFrame
step_dropoff = pd.DataFrame({
    'step_number': range(1, len(sessions_at_least_n) + 1),
    'sessions_remaining': sessions_at_least_n,
    'dropoff_pct': dropoff_pct
})

In [ ]:
# Preview drop-off % at each step... first 20 steps
print(step_dropoff.head(20))

# Plot results for all steps
plt.figure(figsize=(10,6))
plt.plot(step_dropoff['step_number'], step_dropoff['sessions_remaining'], marker='o')
plt.title('User Survival Curve: Sessions Remaining at Each Step')
plt.xlabel('Step Number')
plt.ylabel("Sessions That Haven't Ended Yet")
plt.yscale('log')
plt.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

**Note:** A **log scale** was applied to the y-axis here because most y-axis values are small. The *very few* large y-axis values would dominate this plot without the log application.

**Results:** **Over 50%** of **user sessions** conclude at the end of step 1. Most user sessions are rather short. Extended user sessions are relatively rare, but as seen in Purchase Rate by Pair Length Bucket, the **longer** a session, the more likely it will **result in a purchase**.

### Cohort Analysis... additional columns import from BigQuery

After the initial export to Tableau, the "big picture" analysis fell short, in my opinion. Therefore, 4 new columns were extracted (plus the primary key column) from the raw dataset in BigQuery. This slices the notebook's primary dataset into **4 separate cohorts** based on **how users accessed the site**. The import will take place in the next cell below.

In [ ]:
# Load data
google_2 = pd.read_csv("google_merchandise_cohort_columns.csv")

In [ ]:
# Rename columns for clarity
google_2 = google_2.rename(columns={
    'visitId': 'session_id',
    'pagePath': 'page_path',
    'pagePathLevel1': 'page_path_entry'
})  

In [ ]:
# Remove duplicate rows
google_2_entry = google_2.drop_duplicates(subset='session_id', keep='first')

In [ ]:
# Merge with session_paths (session-level data already here in the Jupyter notebook)
final_merge = session_paths.merge(google_2_entry, on='session_id', how='left')

In [ ]:
# Preview final merged dataset
final_merge.head(10)

In [ ]:
# Number of rows and columns
print(final_merge.shape)

In [ ]:
# Confirm number of unique sessions
unique_sessions_final = final_merge['session_id'].nunique()
print(f'Number of Unique Sessions: {unique_sessions}')

### Purchase Rate by Traffic Source

In [ ]:
# Calculate purchase rate by traffic source
purchase_by_source = (
    final_merge.groupby('source').agg(
        sessions=('session_id', 'count'), purchases=('purchased', 'sum')).assign(
        purchase_rate=lambda x: x['purchases'] / x['sessions']).reset_index()
)

In [ ]:
# Summary stats for number of sessions
summary = purchase_by_source['sessions'].describe()
print(summary.apply(lambda x: f'{x:,.0f}'))

In [ ]:
# Filter for top 20 traffic sources with 100+ sessions
purchase_by_source_filtered = purchase_by_source[purchase_by_source['sessions'] >= 100]

# Display top 20 traffic sources by purchase rate
print(purchase_by_source_filtered.sort_values('purchase_rate', ascending=False).head(20))

### Purchase Rate by Traffic Medium

In [ ]:
# Calculate purchase rate by traffic medium
purchase_by_medium = (
    final_merge.groupby('medium').agg(
        sessions=('session_id', 'count'), purchases=('purchased', 'sum')).assign(
            purchase_rate=lambda x: x['purchases'] / x['sessions']).reset_index()
)

In [ ]:
# Filter for top traffic mediums with 100+ sessions
purchase_by_medium_filtered = purchase_by_medium[purchase_by_medium['sessions'] >= 100]

# Display top traffic mediums by purchase rate
print(purchase_by_medium_filtered.sort_values('purchase_rate', ascending=False))

### Purchase Rate by Traffic Source & Traffic Medium

In [ ]:
# Calculate purchase rate by both traffic source and traffic medium
purchase_by_source_and_medium = (
    final_merge.groupby(['source', 'medium']).agg(
        sessions=('session_id', 'count'), purchases=('purchased', 'sum')).assign(
            purchase_rate=lambda x: x['purchases'] / x['sessions']).reset_index()
)

In [ ]:
# Filter for top traffic source and medium combinations with 100+ sessions
purchase_by_source_and_medium_filtered = purchase_by_source_and_medium[purchase_by_source_and_medium['sessions'] >= 100]

# Display top 15 combinations by purchase rate
purchase_by_source_and_medium_filtered.sort_values('purchase_rate',ascending=False).head(15)

### Purchase Rate by Entry Page

In [ ]:
# Calculate purchase rate by both traffic source and traffic medium
purchase_by_entry_page = (
    final_merge.groupby('page_path_entry').agg(
        sessions=('session_id', 'count'), purchases=('purchased', 'sum')).assign(
            purchase_rate=lambda x: x['purchases'] / x['sessions']).reset_index()
)

In [ ]:
# Filter for top entry pages with 100+ sessions
purchase_by_entry_page_filtered = purchase_by_entry_page[purchase_by_entry_page['sessions'] >= 100]

# Display top entry pages by purchase rate
print(purchase_by_entry_page_filtered.sort_values('purchase_rate', ascending=False))

In [ ]:
# List of checkout-related pages to exclude from results
checkout_pages = [
    '/yourinfo.html',
    '/payment.html',
    '/myaccount.html',
    '/registersuccess.html',
    '/basket.html',
    '/store.html',
    '/register.html',
    '/myaccount.html?mode=billingaddress'
]

# Filter the DataFrame
non_checkout = purchase_by_entry_page_filtered[~purchase_by_entry_page_filtered['page_path_entry'].isin(checkout_pages)]

# Display results... top 10 by purchase rate
print(non_checkout.sort_values('purchase_rate', ascending=False).head(10))

**Note:** The **entry page** is the *very first page* a user lands on when starting a session on the website.
> For example, if a user clicks a Google advertisement for a particular product and lands on "/product/widget.html," then this is the entry page for their session.

## 4 Cohorts based on Traffic Medium

In [ ]:
# Assign cohorts
def medium_cohort(m):
    if m == '(none)':
        return 'Direct'
    elif m in ['cpm', 'cpc']:
        return 'Paid'
    elif m == 'organic':
        return 'Organic'
    elif m == 'referral':
        return 'Referral'
    else:
        return 'Other'

final_merge['medium_cohort'] = final_merge['medium'].apply(medium_cohort)

In [ ]:
# Calculate sessions, purchases, and purchase rate for all cohorts
cohort_summary = (
    final_merge.groupby('medium_cohort').agg(
        sessions=('session_id', 'nunique'), purchases=('purchased', 'sum')).assign(
            purchase_rate=lambda x: x['purchases'] / x['sessions']).reset_index()
)

In [ ]:
# Display cohort summary table
cohort_summary.sort_values('purchase_rate', ascending=False)

In [ ]:
# Plot results
plt.bar(cohort_summary['medium_cohort'], cohort_summary['purchase_rate'])
plt.title('Purchase Rate by Cohort')
plt.xlabel('Cohort')
plt.ylabel('Purchase Rate')
plt.show()

#### Cohort Descriptions

- **Direct:**
  Users who arrived by typing the website URL *directly* into their browser, using bookmarks, or in ways that bypass any tracked referring site.

- **Paid:**
  Users who clicked on paid advertisements, such as search engine ads (CPC, CPM) or display ads.

- **Organic:**
  Users who found the site via unpaid search results, such as Google or Bing.

- **Referral:**
  Users who accessed the site by clicking a link on a different (non-affiliated) website.

- **Other:**
  Any remaining traffic not captured by the four main categories above, often including miscategorized, rare, or unknown traffic sources. (This will be filtered out in the final Tableau dashboard.)

### Drop-Off % at Each Step... broken down by each cohort

In [ ]:
# This DataFrame will be recreated with an appended cohort column
step_dropoff

In [ ]:
# Extract cohort labels from the final merged dataset
cohorts = final_merge['medium_cohort'].unique()

In [ ]:
# Loop through each cohort and compute the drop-off
max_length_ = final_merge['path_length'].max()
max_steps = 500
step_numbers = list(range(1, max_length_ + 1))
results = []

for cohort in cohorts:
    df = final_merge[final_merge['medium_cohort'] == cohort]
    session_lengths_ = df.groupby('session_id')['path_length'].max()
    
    sessions_at_least_n_ = [(session_lengths_ >= n).sum() for n in range(1, max_length_ + 1)]
    dropoff_pct_ = [
        1 - (sessions_at_least_n_[n] / sessions_at_least_n_[n-1]) if n > 0 and sessions_at_least_n_[n-1] > 0 else 0 for n in range(len(sessions_at_least_n_))
    ]

    assert len(sessions_at_least_n_) == max_steps
    assert len(dropoff_pct_) == max_steps
    
    # Create cohort df
    cohort_info = pd.DataFrame({
        'step_number': step_numbers,
        'sessions_remaining_': sessions_at_least_n_,
        'dropoff_pct': dropoff_pct_,
        'cohort': [cohort] * max_steps
    })
    results.append(cohort_info)

In [ ]:
# Concatenate and display dropoff % by step (with cohort labels now)
cohort_funnel = pd.concat(results, ignore_index=True)
cohort_funnel

### Final Export to Tableau

In [ ]:
# Save the two primary DataFrames as CSVs
cohort_funnel.to_csv('dropoff_by_cohort.csv', index=False)
final_merge.to_csv('final_merged_dataset.csv', index=False)

These two CSVs will be opened in Tableau to create the final dashboard.